By default the latest version of the API and the latest supported Spark version is chosen. To specify your own: %use spark-streaming(spark=3.2, v=1.1.0)

In [2]:
%use spark-streaming

To start a spark streaming session, simply use `withSparkStreaming { }` inside a cell. To use Spark normally, use `withSpark { }` in a cell, or use `%use spark` to start a Spark session for the whole notebook.


Let's define some data class to work with.

In [4]:
data class TestRow(
    val word: String,
)

To run this on your local machine, you need to first run a Netcat server: `$ nc -lk 9999`.

This example will collect the data from this stream for 10 seconds and 1 second intervals, splitting and counting the input per word.

In [5]:
withSparkStreaming(batchDuration = Durations.seconds(1), timeout = 10_000) { // this: KSparkStreamingSession

    val lines: JavaReceiverInputDStream<String> = ssc.socketTextStream("localhost", 9999)
    val words: JavaDStream<String> = lines.flatMap { it.split(" ").iterator() }

    words.foreachRDD { rdd: JavaRDD<String>, _: Time ->
        withSpark(rdd) { // this: KSparkSession
            val dataframe: Dataset<TestRow> = rdd.map { TestRow(it) }.toDS()
            dataframe
                .groupByKey { it.word }
                .count()
                .show()
        }
    }
}

+---+--------+
|key|count(1)|
+---+--------+
+---+--------+

+-----+--------+
|  key|count(1)|
+-----+--------+
|hello|       8|
|Hello|       6|
|world|       3|
|     |       2|
| test|       4|
+-----+--------+

+-----+--------+
|  key|count(1)|
+-----+--------+
|hello|       3|
+-----+--------+

+---+--------+
|key|count(1)|
+---+--------+
+---+--------+

+---+--------+
|key|count(1)|
+---+--------+
+---+--------+

+---+--------+
|key|count(1)|
+---+--------+
+---+--------+

+---+--------+
|key|count(1)|
+---+--------+
+---+--------+

+---+--------+
|key|count(1)|
+---+--------+
+---+--------+

+-----+--------+
|  key|count(1)|
+-----+--------+
|hello|       1|
|world|       2|
+-----+--------+

+---+--------+
|key|count(1)|
+---+--------+
+---+--------+

